# 텍스트 마이닝 05 - Word embedding
<img width="40%" src="https://i.imgur.com/AydcsdT.png">
    
### 2017-2019 [FinanceData.KR](http://financedata.kr)

# 환경설정

In [31]:
# 폰트설치 (구글 Colab)
!apt-get install -q fonts-nanum*

Reading package lists...
Building dependency tree...
Reading state information...
fonts-nanum is already the newest version (20170925-1).
fonts-nanum-coding is already the newest version (2.5-1).
fonts-nanum-eco is already the newest version (1.000-6).
fonts-nanum-extra is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.


In [0]:
# install nltk, konlpy, gensim
!pip install -q nltk
!pip install -q konlpy
!pip install -q gensim

In [0]:
# FutureWarning: from `int` to `np.signedinteger` is deprecated. In future

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Word embedding
단어들을 벡터 공간에 배치하는 것 (문서로 부터 특징을 학습)
* 단어를 이진법으로 표현: one-hot encoding (예를 들어, [0 0 0 0 0 1 0 0 0] )
* 이진 표현은 유사도를 정의할 수 없다 (모든 벡터간의 거리가 같다)
* TF-IDF: TF(=단어가 문서에 존재 여부) X IDF(=문서 빈도의 역수), 유사도 식별이 어렵다
* local context 를 사용하여, syntactic or semantic 표현 정보로 활용하자!

주요 접근법
* NNLM과 RNNLM
* Neural networks (뉴럴네트워크)
* Dimensionality reduction (차원축소)

# word2vec (Mikolov 2013)
* Word Embeddins To Vector, "단어를 벡터(숫자)공간에 삽입하기"
* word2vec = CBOW + Skip-Gram (-hidden layers)
* 엄청난 성능!

<img width="60%" src="https://i.imgur.com/JdF4YEf.png">

CBOW |  Skip-Gram
:---- | :----
ㆍ 문맥(전후 2개 단어)로 단어를 예측 <br> ㆍ Skip-Gram 학습에 비해 매우 빠르다 | ㆍ 현재 단어를 통해 주위 문맥을 예측<br> ㆍ 작은 량의 학습 데이터로도 잘 동작

# Linguistic regularities
#### KINGS - KING + QUEEN = ?
<img src="https://i.imgur.com/SI3Ey0u.png" >


# 단어의 벡터 연산 예
http://w.elnn.kr/ (여기서 연습해보세요)

v(KING) - v(MAN) + v(WOMAN) = v(QUEEN) 
* 남자==왕, 여자=?

v(한국) - v(서울) + v(파리) 
* 서울==한국, 파리=?

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import font_manager
font_path = '/usr/share/fonts/truetype/nanum/NanumMyeongjoBold.ttf'
font_name = font_manager.FontProperties(fname=font_path).get_name()

plt.rcParams["font.family"] = font_name
plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = (14,4)

# 증권 뉴스 코퍼스

2016년 1년간 전종목 증권뉴스
* 파일 307,926개, 크기 776.8 MB
* https://www.dropbox.com/s/hv73gfwnc2x450z/news_text_2015.zip 

2017년 1월 1개월간 전종목 증권뉴스
* 항목 14,515개, 크기 34.6 MB
* https://www.dropbox.com/s/yw0ug8chelh1leu/news_text_2016_07.zip
    


In [0]:
# 현재 디렉토리에 다운로드
! wget -q https://www.dropbox.com/s/yw0ug8chelh1leu/news_text_2016_07.zip

In [36]:
ll *.zip

-rw-r--r-- 1 root 21369720 Mar 24 18:36 news_text_2016_07.zip


In [37]:
import zipfile

zip_file = 'news_text_2016_07.zip'
zf = zipfile.ZipFile(zip_file, mode='r')
zlist = zf.infolist()

z = zlist[0]
with zf.open(z.filename) as f:
    print(f.read().decode('utf-8'))


유재훈 한국예탁결제원 사장/사진제공=한국예탁결제원한국예탁결제원이 전자증권제 조기 안착을 위해 홍보와 마케팅에 주력할 방침이다. 유재훈 한국예탁결제원 사장은 11일 서울 여의도에서 열린 기자간담회에서 “전자증권제가 선진국 등 다른 나라에 비해 도입이 늦어졌지만, 조기 시행을 건의해 제도 홍보와 관련 시스템 구축에 최선을 다하겠다”고 밝혔다. 전자증권제도는 실물증권에 기반해 이뤄지던 증권의 발행과 유통을 전자 등록을 통해 추진하는 시스템이다. OECD 34개 국가 중 32개국이 일부 또는 전면 도입했으며, 국내에는 지난 3월 22일 전자증권법 제정법률이 국회에서 통과됐다. 전자증권법은 법상 시행일이 공포 후 4년 이내지만, 한국예탁결제원은 조기시행될 수 있도록 건의할 계획이라고 설명했다. 유 사장은 “20년간 추진해왔던 전자증권법은 장기 숙원과제였다”며 “연간 870억원의 경제적 효과가 발생할 것”이라고 강조했다. 이 외에도 한국예탁결제원은 글로벌 법인식별기호(LEI) 활용과 의무사용이 확대됨에 따라 글로벌LEI재단의 국제 인증도 추진할 예정이다. 또 크라우드넷을 활용해 크라우드펀딩 정보허브로 육성하고 KSD지방사무소를 중심으로 중소·벤처기업 대상 주식사무와 크라우드펀딩 연수에도 총력을 기울일 방침이다. 또한 중국 선전과 홍콩 증시의 교차거래가 허용되는 선강퉁(深港通) 시행에 대비해 거래주식의 예탁결제 서비스를 준비하는 등 중국 자본시장에 대한 투자지원 서비스를 강화할 계획이다.유 사장은 “임기가 4개월 가량 남았지만, 신성장 동력 확보와 글로벌화 추진 등 4대 전략을 차근차근 이행할 것”이라며 “자본시장법 개정안의 국회통과를 적극 지원함으로 거래소와 지배관계도 차근차근 해소하겠다”고 말했다. /박시진기자 see1205@sedaily.com[서울경제 바로가기] [인기포토로 보는 오늘]서울경제와 친해지는 가장 확실한 방법 [페이스북]저작권자 ⓒ 서울경제, 무단 전재 및 재배포 금지서울경제 관련뉴스언론사 페이지로 이동합니다.이승연 남편 김문철 "이승연, 부모님보다 

In [38]:
len(zlist)

14515

# 1. Read news text

In [39]:
import zipfile
zip_file = 'news_text_2016_07.zip' # 항목 14,515개, 크기 34.6 MB

zf = zipfile.ZipFile(zip_file, mode='r')
zlist = zf.infolist()

news_text_list = []
for z in zlist:
    with zf.open(z.filename) as f:
        news_text_list.append( f.read().decode('utf-8') )
        
        
len(news_text_list)

14515

# 2. Tokenize
Mecab를 사용하려면 별도 설치 필요

Mecab 설치, https://nbviewer.jupyter.org/e208aeeaa2c8ba80e46a124d5056e267

In [40]:
%%time
from konlpy.tag import Okt, Kkma, Hannanum, Komoran, Mecab

engine = Mecab()

news_tokens = []
for news_text in news_text_list:
    nouns = engine.nouns(news_text)
    nouns = [n for n in nouns if len(n) > 1]
    news_tokens.append(nouns)

CPU times: user 36.5 s, sys: 109 ms, total: 36.6 s
Wall time: 36.6 s


In [41]:
print(news_tokens[0][:50])

['유재훈', '한국', '예탁', '결제', '사진', '제공', '한국', '예탁', '결제', '한국', '예탁', '결제', '전자', '증권', '조기', '안착', '홍보', '마케팅', '주력', '방침', '유재훈', '한국', '예탁', '결제', '사장', '서울', '여의도', '기자', '간담회', '전자', '증권', '선진국', '나라', '도입', '조기', '시행', '건의', '제도', '홍보', '관련', '시스템', '구축', '최선', '전자', '증권', '제도', '실물', '증권', '기반', '증권']


In [42]:
len(news_tokens)

14515

# 3. Training

In [0]:
from gensim.models.word2vec import Word2Vec

# size=100 (100차원 벡터), window=10 (10단어씩), min_count=3 (최소 3회 이상 출현)
model = Word2Vec(news_tokens, size=100, window=10, min_count=3)

In [77]:
# word counts
len(model.wv.vocab)

16501

In [87]:
print(list(model.wv.vocab.keys())[:100])

['유재훈', '한국', '예탁', '결제', '사진', '제공', '전자', '증권', '조기', '안착', '홍보', '마케팅', '주력', '방침', '사장', '서울', '여의도', '기자', '간담회', '선진국', '나라', '도입', '시행', '건의', '제도', '관련', '시스템', '구축', '최선', '실물', '기반', '발행', '유통', '등록', '추진', '국가', '개국', '일부', '전면', '국내', '제정법', '국회', '통과', '법상', '공포', '이내', '계획', '설명', '장기', '숙원', '과제', '연간', '경제', '효과', '발생', '강조', '글로벌', '법인', '식별', '기호', '활용', '의무', '사용', '확대', '재단', '국제', '인증', '예정', '우드', '펀딩', '정보', '허브', '육성', '지방', '사무소', '중심', '중소', '벤처', '기업', '대상', '주식', '사무', '크라', '연수', '총력', '중국', '선전', '홍콩', '증시', '교차', '거래', '허용', '선강', '대비', '서비스', '준비', '자본', '시장', '투자', '지원']


In [58]:
# 삼성전자 벡터값 (100개)
model.wv['삼성전자']

array([-1.7933525 ,  0.36936912,  0.08875873,  0.59603745, -3.147714  ,
        0.09319665, -0.01413262,  2.1011016 , -0.88297397, -0.70087457,
       -2.391501  , -2.0659525 ,  2.8617103 , -3.370025  , -2.977699  ,
       -0.3832729 , -2.2520823 ,  3.6736243 , -4.2244334 , -2.1860306 ,
       -2.5353246 ,  3.2167165 ,  0.4663929 ,  1.4055012 , -0.16936238,
       -4.0087266 , -5.47021   ,  6.2920194 ,  3.282837  , -1.0415668 ,
       -2.8230345 , -2.0974655 ,  4.834671  ,  2.0735846 ,  3.2560494 ,
       -1.43623   , -0.5091899 , -0.29265332, -3.1176982 ,  0.3212348 ,
       -2.508291  ,  1.5678276 , -2.036721  , -3.5913355 , -1.1142409 ,
        1.352353  , -0.14999886, -3.4449966 ,  0.667729  ,  0.46756113,
       -1.5068873 , -2.2653942 , -0.3418202 , -1.0173693 , -1.0548853 ,
       -3.3327332 ,  3.0920737 , -1.3088682 , -4.923753  , -1.1171741 ,
       -1.6504915 ,  0.0329872 ,  3.5654914 ,  1.0923164 , -0.4592032 ,
        2.7190323 ,  4.585735  , -2.1466308 , -1.0868313 ,  6.75

# 4. Testing

In [59]:
model.wv.most_similar('삼성전자') # 단어의 의미 (종목)

[('하이닉스', 0.571800947189331),
 ('서프라이즈', 0.5095070600509644),
 ('장주', 0.4951897859573364),
 ('최고가', 0.4881258010864258),
 ('한국전력', 0.48707282543182373),
 ('중형주', 0.4827226400375366),
 ('삼성물산', 0.47940513491630554),
 ('경신', 0.4644964337348938),
 ('디스플레이', 0.46217405796051025),
 ('고가', 0.45499318838119507)]

In [60]:
model.wv.most_similar('코스피') # 단어의 의미 (시장)

[('지난주', 0.6485657095909119),
 ('보합권', 0.6160129308700562),
 ('장세', 0.6130313873291016),
 ('랠리', 0.6113300323486328),
 ('매수세', 0.6068103909492493),
 ('증시', 0.606791615486145),
 ('양봉', 0.6028212308883667),
 ('지수', 0.5960755348205566),
 ('초반', 0.5700454711914062),
 ('고점', 0.5616500377655029)]

In [61]:
model.wv.most_similar('반도체') # 단어의 의미 (산업)

[('낸드', 0.6747739315032959),
 ('장비', 0.6654652953147888),
 ('디스플레이', 0.632972240447998),
 ('메모리', 0.608782947063446),
 ('도시바', 0.5721738934516907),
 ('다이오드', 0.5647817850112915),
 ('플래시', 0.5589834451675415),
 ('발광', 0.5521066188812256),
 ('마이크', 0.5462299585342407),
 ('유기', 0.5451456308364868)]

In [62]:
model.wv.most_similar('스마트폰') # 단어의 의미 (제품)

[('갤럭시', 0.6402682065963745),
 ('파란색', 0.6402331590652466),
 ('메모리', 0.63810133934021),
 ('애플', 0.6299062967300415),
 ('팬택', 0.6193400621414185),
 ('소니', 0.6186425685882568),
 ('카메라', 0.6096933484077454),
 ('중저가', 0.5975085496902466),
 ('플래시', 0.5951032638549805),
 ('모바일', 0.5826479196548462)]

In [63]:
model.wv.most_similar(positive=['바이오'])

[('녹십자', 0.7266756296157837),
 ('제약', 0.7200186252593994),
 ('신약', 0.679617166519165),
 ('약품', 0.6163440346717834),
 ('약주', 0.6145680546760559),
 ('제약사', 0.6095517873764038),
 ('효종', 0.6083720922470093),
 ('보령', 0.5932897329330444),
 ('랜드', 0.5913833975791931),
 ('대웅', 0.5644354820251465)]

In [64]:
model.wv.most_similar(positive=['삼성전자', '자동차'], negative=['스마트폰'])

[('기아차', 0.5292428135871887),
 ('현대차', 0.5206381678581238),
 ('만도', 0.48896604776382446),
 ('기아', 0.459636926651001),
 ('해상', 0.427784264087677),
 ('과대', 0.427043080329895),
 ('공기업', 0.4262397289276123),
 ('페라', 0.42400866746902466),
 ('체면', 0.4127246141433716),
 ('현대기업금융', 0.40234360098838806)]

In [65]:
model.wv.most_similar(positive=['스마트폰', '전기차'], negative=['액정'])

[('애플', 0.5284238457679749),
 ('구글', 0.5176868438720703),
 ('소니', 0.5000623464584351),
 ('아이폰', 0.49766427278518677),
 ('모바일', 0.49708041548728943),
 ('정면', 0.4864993989467621),
 ('오하', 0.4776430130004883),
 ('팬택', 0.4757494330406189),
 ('배터리', 0.475460946559906),
 ('통사', 0.4675852656364441)]

# 가장 관련이 없는 것은?

In [66]:
model.wv.doesnt_match("스마트폰 배터리 액정 맥주 아이폰 갤럭시".split())

'맥주'

In [67]:
model.wv.doesnt_match("바이오 시밀러 제약 코스피 코스닥".split())

'코스피'

# 유사도(similarity) 측정 

In [68]:
model.wv.similarity('스마트폰', '배터리')

0.39509022

In [69]:
model.wv.similarity('전기차', '배터리')

0.74210894

In [89]:
print('삼성전자', '하이닉스', model.wv.similarity('삼성전자', '하이닉스'))
print('삼성전자', '녹십자', model.wv.similarity('삼성전자', '녹십자'))
print('삼성전자', '갤럭시', model.wv.similarity('삼성전자', '갤럭시'))
print('삼성전자', '스마트폰', model.wv.similarity('삼성전자', '스마트폰'))
print('삼성전자', '농심', model.wv.similarity('삼성전자', '농심'))

삼성전자 하이닉스 0.571801
삼성전자 녹십자 -0.0034018178
삼성전자 갤럭시 0.44114164
삼성전자 스마트폰 0.44459933
삼성전자 농심 0.002165698


# Pros and cons

PROS
* 사용 간편, 노력대비 엄청난 효과
* 라벨 없는 데이터로 Supervied Learning

CONS
* 정확한 설명이 어려움 (머신러닝의 특징)
* 성능향상을 위해 많은 튜닝 필요

# Summary
* Word embedding and word2vec
* 단어의 벡터 연산
* 증권 뉴스 코퍼스
* word2vec 텍스트 마이닝
 * 1) Read text -- ZipFile
 * 2) Tokenize -- e.nouns(text)
 * 3) Training -- Word2Vec()
 * 4) Tesging -- model.most_similar(positive=[], negative=[]) 
* 가장 관련이 없는 것은? - model.wv.doesnt_match([])
* 유사도 측정 - model.wv.similarity(w1, w2)
* Pros and cons

#### 2017-2019 FinanceData.KR http://financedata.kr